In [1]:
import scipy.io as scio
import os
import mne
import numpy as np
import time
from IPython import display
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import matplotlib.image as mpimg
import pandas as pd
import seaborn as sns
from mne.time_frequency import tfr_multitaper
from mne.stats import permutation_cluster_1samp_test as pcluster_test
import scipy.stats as stats
from statannotations.Annotator import Annotator
%matplotlib inline
# sns.set(style="whitegrid")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/tmp/ipykernel_22719/3243943048.py:21: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def delete_multiple_elements(list_object, indices):
    indices = sorted(indices, reverse=True)
    for idx in indices:
        if idx < len(list_object):
            list_object.pop(idx) # define functions for extracting relevant epochs
            
def stats_plot(df, save_dir, feature, file_name):
    '''
    df = dataframe (mean, peak)
    save_dir = directory to save stats PNG and TXT files
    feature = 'mean','peak'
    '''    

    pairs=[((f,conditions[0]),(f,conditions[1])) for f in freq_bands_of_interest]

    for roi in roi_names:        
        df_ch_tmp=df.query(f"channel=='{roi}'")
       

        #stats plot
        plt.figure(figsize=(8.6,6.5));
        ax_tmp = sns.boxplot(data=df_ch_tmp,x='band', y='value', hue='condition',
                palette='pastel', order=freq_bands_of_interest,
                hue_order=conditions,
                linewidth=0.5)

        ax_tmp.set_title(f"{roi} | Resting state Power | Feature: '{feature}' ",pad=30)

        #stats output
        x='band'
        y='value'
        hue='condition'
        order=freq_bands_of_interest
        hue_order = conditions

        test = 'Mann-Whitney'
        print(f"******************************************************{roi} {test}******************************************************")
        annot = Annotator(ax_tmp, pairs, data=df_ch_tmp, x=x, y=y, hue=hue, order=order,hue_order=hue_order)
        annot.configure(test=test, text_format='star', loc='outside',comparisons_correction=None)
        annot.apply_test()
        annot.annotate()

        # save plot and tfr data
        # TODO: follow same naming convention as in preprocessing code
        # Add region to end, removed raw.fif
        save_fname = f"{file_name[:-4]}_{roi}"
        
        # plot as png
        plt.savefig(os.path.join(save_dir,sub_folder,save_fname+".png"))
        plt.figure()
        
        # tfr data as csv
        df_ch_tmp.to_csv(os.path.join(save_dir,sub_folder,save_fname+".csv"))
        display.clear_output(wait=True)  

PSD Generation and Calculation Functions

In [4]:
# File pair generator

def file_pair_generator(file_dir, sub_id, sub_state):
    same_state_double_pairs = []
    day_1_data = []
    day_last_data = []
    for file in os.listdir(file_dir):
        if str(sub_id) in file and str(sub_state) in file and not '512' in file:
            same_state_double_pairs.append(file)
    for i in range(0, len(same_state_double_pairs)):
        if 'Day 1' in same_state_double_pairs[i]:
            day_1_data.append(same_state_double_pairs[i])
        else:
            day_last_data.append(same_state_double_pairs[i])
    return day_1_data, day_last_data
            
            

In [5]:
# Functions are divided into surface and depth generators, surface generator allows users to select chosen nodes given image data
# depth generator used for when images can offer no better info on channel location

def ROI_Surface_Generator(ROI_Data_Path, ROI_sub_id, surface_side): # surface side must be 'lh' (lefthand) or 'rh' (righthand)
    for file in os.listdir(ROI_Data_Path):
        if str(ROI_sub_id) in file and not str('depth') in file and str(surface_side) in file:
            processed_roi_csv = pd.read_csv(os.path.join(ROI_Data_Path, file))

    ROI_channels = processed_roi_csv['ID'].to_list()
    Chosen_ROI_Channels = [] # reset
    channel_ids = 0
    
    while channel_ids != len(ROI_channels):
        verification_status = input(f"Proposed Channel in Region of Interest: {ROI_channels[channel_ids]} - Verify from Image, Keep y/n ?")
        if verification_status == 'y':
            Chosen_ROI_Channels.append(ROI_channels[channel_ids])
            channel_ids = channel_ids + 1
            display.clear_output(wait=True)
        elif verification_status == 'n':
            channel_ids = channel_ids + 1
            display.clear_output(wait=True)
        else:
            print("Invalid Reponse, please enter 'y' or 'n'")
            channel_ids = channel_ids + 0
            display.clear_output(wait=True)
            
    return(Chosen_ROI_Channels) # returns chosen channels as list of channel IDs



def ROI_Depth_Generator(ROI_Data_Path, ROI_sub_id): # use for files containing 'depth'
    for file in os.listdir(ROI_Data_Path):
        if str(ROI_sub_id) in file and str('depth') in file:
            processed_roi_csv = pd.read_csv(os.path.join(ROI_Data_Path, file))
    
    ROI_channels = processed_roi_csv['ID'].to_list()
    return(ROI_channels)

In [6]:
# Automate the generation of broad or MUA (>100 Hz) psds and combine them

def notch_filter_analysis(broad_or_MUA, generated_epochs, avg_psd_broad_lower, picks): # enter 'broad' or 'MUA' for broad_or_MUA argument
    if broad_or_MUA == 'broad':
        broad_band_list = [] # reset
        for i in range(0, (int(MUA_max/60)-1)):
            if i == 0:
                broad_band = generated_epochs.compute_psd(fmin=65., fmax=117., picks=picks, n_jobs=n_cores)
                avg_psd_broad_band = np.mean(broad_band, axis=2)
                broad_band_list.append(avg_psd_broad_band)
            elif i == (int(MUA_max/60)-1):
                broad_band = generated_epochs.compute_psd(fmin=63+i*60., fmax = MUA_max, picks = picks, n_jobs=n_cores)
                avg_psd_broad_band = np.mean(broad_band, axis=2)
                broad_band_list.append(avg_psd_broad_band)
            else:
                broad_band = generated_epochs.compute_psd(fmin=63+i*60., fmax=57+(i+1)*60., picks=picks, n_jobs=n_cores)
                avg_psd_broad_band = np.mean(broad_band, axis=2)
                broad_band_list.append(avg_psd_broad_band)
        broad_band_list.append(avg_psd_broad_lower)
        avg_psd_broad = np.mean(np.array(broad_band_list), axis=0)
        return(avg_psd_broad)
    elif broad_or_MUA == 'MUA':
        MUA_band_list = [] # reset
        for i in range(0, (int(MUA_max/60)-1)):
            if i == 0:
                MUA_band = generated_epochs.compute_psd(fmin=100., fmax = 117., picks=picks, n_jobs=n_cores)
                avg_psd_MUA_band = np.mean(MUA_band, axis=2)
                MUA_band_list.append(avg_psd_MUA_band)
            elif i == int(MUA_max/60-1):
                MUA_band = generated_epochs.compute_psd(fmin=63+i*60, fmax = MUA_max, picks=picks, n_jobs=n_cores)
                avg_psd_MUA_band = np.mean(MUA_band, axis=2)
                MUA_band_list.append(avg_psd_MUA_band)
            else:
                MUA_band = generated_epochs.compute_psd(fmin = 63+i*60., fmax = 117+i*60., picks=picks, n_jobs=n_cores)
                avg_psd_MUA_band = np.mean(MUA_band, axis=2)
                MUA_band_list.append(avg_psd_MUA_band)
        avg_psd_MUA = np.mean(np.array(MUA_band_list), axis=0)
        return(avg_psd_MUA)
    

In [7]:
def avg_psd_arr_generator(fif_file_path, file_pair, picks):
    avg_psd_arr_lst = [] # reset
    for i in range(0, len(file_pair)):
        raw = mne.io.read_raw_fif(os.path.join(fif_file_path, str(file_pair[i]))).crop(tmin, tmax).load_data()
        print(f"processing data for {file_pair[i]}")
        raw = raw.notch_filter(np.arange(120, 481, 60), n_jobs=-1, fir_design='firwin', notch_widths = 6) # inital notch filter for all harmonics
        epochs = mne.make_fixed_length_epochs(raw, duration=30, preload=False)
        n_fft = 2048  # the FFT size (n_fft). Ideally a power of 2

    # Calculate the psds from raw data based on frequency range
        broad1 = epochs.compute_psd(fmin=1., fmax=55., picks=picks, n_jobs=n_cores)
        delta = epochs.compute_psd(fmin=1., fmax=4., picks=picks, n_jobs=n_cores)
        theta = epochs.compute_psd(fmin=4., fmax=8., picks=picks, n_jobs=n_cores)
        alpha = epochs.compute_psd(fmin=8., fmax=13., picks=picks, n_jobs=n_cores)
        beta = epochs.compute_psd(fmin=13., fmax=30., picks=picks, n_jobs=n_cores)
        low_gamma = epochs.compute_psd(fmin=30., fmax=55., picks=picks, n_jobs=n_cores)
        high_gamma = epochs.compute_psd(fmin=65., fmax=100., picks=picks, n_jobs=n_cores)
    
    
    # Get average power across bands (shape = epochs x channels)
        avg_psd_broad1 = np.mean(broad1, axis=2)
        avg_psd_delta = np.mean(delta, axis=2)
        avg_psd_theta = np.mean(theta, axis=2)
        avg_psd_alpha = np.mean(alpha, axis=2)
        avg_psd_beta = np.mean(beta, axis=2)
        avg_psd_low_gamma = np.mean(low_gamma, axis=2)
        avg_psd_high_gamma = np.mean(high_gamma, axis=2)
        
    # Calculate the average MUA and larger broad frequencies using the notch filter analysis function
        avg_psd_broad = notch_filter_analysis('broad', epochs, avg_psd_broad1, picks)
        avg_psd_MUA = notch_filter_analysis('MUA', epochs, avg_psd_broad1, picks) # Final argument is neccecary for funtion, but plays no part in MUA calculations
        display.clear_output()
        print(f"applying notch filter across all epochs for broad2 analysis and MUA frequencies")


    # Combine broad spectrum powers and take the trace
        trace_avg_psd_broad = np.trace(avg_psd_broad)
        display.clear_output()
        print(f"Trace of the average broad spectrum power: {trace_avg_psd_broad}")    

    # Get band PSD's normalized by the trace of the broad spectrum and convert to dB
        avg_psd_delta = 10 * np.log10(avg_psd_delta / trace_avg_psd_broad)
        avg_psd_theta = 10 * np.log10(avg_psd_theta / trace_avg_psd_broad)
        avg_psd_alpha = 10 * np.log10(avg_psd_alpha / trace_avg_psd_broad)
        avg_psd_beta = 10 * np.log10(avg_psd_beta / trace_avg_psd_broad)
        avg_psd_low_gamma = 10 * np.log10(avg_psd_low_gamma / trace_avg_psd_broad)
        avg_psd_high_gamma = 10 * np.log10(avg_psd_high_gamma / trace_avg_psd_broad)
        avg_psd_MUA = 10 * np.log10(avg_psd_MUA / trace_avg_psd_broad)

        avg_psd_arr = np.array((avg_psd_delta, avg_psd_theta, 
                            avg_psd_alpha, avg_psd_beta,
                            avg_psd_low_gamma, avg_psd_high_gamma, avg_psd_MUA)) 

        avg_psd_arr_lst.append(avg_psd_arr)

    ID_sub_state_psd_arr = np.concatenate(avg_psd_arr_lst, axis = 0)
    return(ID_sub_state_psd_arr)        




Z-Score Calculation Functions

In [8]:
# Z-SCORE CALCULATION SETTINGS

# 1. Choose Asleep1, Asleep2, Awake1, or Awake2 - Arbitrary from current understanding, doesn't really matter for calculations
zsub_state = 'Asleep1'

# 2. Choose time frame of baseline z-score - VALUES MUST BE FLOAT (decimal at the end)
# IMPORTANT: note that all future time frames must match the baseline in length
ztmin, ztmax = 270., 300.
print(f"Length of time frames is {ztmax - ztmin} seconds")

Length of time frames is 30.0 seconds


In [9]:
# Take the data file given the subject ID and sub_state, to generate baseline time frame
# Uses the time frame selected in Z-Score settings and must be given a specific channel to operate
# Use the ROI generators to find channels of interest

def z_score_baseline(data_dir, ztmin, ztmax, sub_id, zsub_state):
    for files in os.listdir(data_dir):
        if str(sub_id) in files and str(zsub_state) in files and not 'Day 1' in files:
            file_name = files
    raw = mne.mne.io.read_raw_fif(os.path.join(data_dir, file_name))
    raw_df = raw_df[raw_df['time'] > ztmin and raw_df['time'] < ztmax] 
    base_mean = np.mean(raw_df)
    base_std = np.std(raw_df)
    return base_mean, base_std

In [18]:
def z_score_calculator(data_dir, zsub_state, ztmin, ztmax, sub_id, picks):
    file_list = [] # reset
    for file in os.listdir(data_dir):
        if sub_id in file and not '512' in file and str(zsub_state)[:-1] in file:
            file_list.append(file)
            print(f"file added to analyzing list: {file}")
            
    z_score_arr_lst = [] # reset
    
    for l in range(len(file_list)):
        if str(zsub_state) in file_list[l] and not 'Day 1' in file_list[l]:
            raw = mne.io.read_raw_fif(os.path.join(data_dir, file_list[l]))
            raw_df = raw.to_data_frame(picks = picks)
            selected_df = raw_df[raw_df['time'] > ztmin]
            selected_df = selected_df[selected_df['time'] < ztmax] 
            base_mean = np.mean(selected_df)
            base_std = np.std(selected_df)
            display.clear_output()
            print(f"The baseline mean is: {base_mean} | The baseline std is: {base_std}")
            display.clear_output(wait = True)
        
    for k in range(len(file_list)):
        z_score_list = []
        if zsub_state in str(file_list[k]) and not 'Day 1' in str(file_list[k]):
            if ztmin > 0 and ztmax < 300:
                raw_df = raw_df[raw_df['time'] < ztmin]
                raw_df = raw_df[raw_df['time'] > ztmax]
            elif ztmax == 300:
                raw_df = raw_df[raw_df['time'] < (ztmax - (ztmax-ztmin))]
            else:
                raw_df = raw_df[raw_df['time'] > (ztmin + (ztmax-ztmin))]
            
            print(f"Shape of raw data in file: {file_list[k]}: {raw_df.shape}")

            for i in range(1, (raw_df.shape[1])):
                column = raw_df.iloc[:,i].to_list()
                z_scores = [] # reset
                for j in range(0, len(column)):
                    z_score = (column[j] - base_mean[i]) / base_std[i]
                    z_scores.append(z_score)
            final_day_z_score_arr = np.array(z_score_list)        
            print(f"Process #{k}, for file: {file_list[k]}, shape is {z_score_arr.shape}")

        else:
            raw = mne.io.read_raw_fif(os.path.join(data_dir, str(file_list[k])))
            raw_df = raw.to_data_frame(picks = picks)
            print(f"Shape of raw data in file: {file_list[k]}: {raw_df.shape}")
            for i in range(1, (raw_df.shape[1])):
                column = raw_df.iloc[:,i].to_list()
                z_scores = [] # reset
                for j in range(0, len(column)):
                    z_score = (column[j] - base_mean[i]) / base_std[i] 
                    z_scores.append(z_score)
                z_score_list.append(z_scores)
            z_score_arr = np.array(z_score_list)
            z_score_arr_lst.append(z_score_arr)
            print(f"Process #{k}, for file: {file_list[k]}, shape is {z_score_arr.shape}")
            
    z_score_full_arr = np.stack(z_score_arr_lst, axis=-1) # stacks the individual z score arrays, with 3rd dimension representing each array
    return(z_score_full_arr, final_day_z_score_arr) # returns two separate arrays because array for calculating bas
    
            
        
    

In [19]:
z_score_test, z_score_train = z_score_calculator(processed_info_dir, zsub_state, ztmin, ztmax, sub_ids_lst[0], selected_picks)

Opening raw data file ../../../George Kenefati/Dan Friedman sEEG Data/Processed FIF Data/NY846_Day 1_Asleep2-raw.fif...
Isotrak not found
    Range : 0 ... 615039 =      0.000 ...   300.312 secs
Ready.
Shape of raw data in file: NY846_Day 1_Asleep2-raw.fif: (615040, 14)
Process #0, for file: NY846_Day 1_Asleep2-raw.fif, shape is (13, 615040)
Shape of raw data in file: NY846_Day 8_Asleep1-raw.fif: (552960, 14)
Process #1, for file: NY846_Day 8_Asleep1-raw.fif, shape is (13, 615040)
Opening raw data file ../../../George Kenefati/Dan Friedman sEEG Data/Processed FIF Data/NY846_Day 1_Asleep1-raw.fif...
Isotrak not found
    Range : 0 ... 617087 =      0.000 ...   301.312 secs
Ready.
Shape of raw data in file: NY846_Day 1_Asleep1-raw.fif: (617088, 14)
Process #2, for file: NY846_Day 1_Asleep1-raw.fif, shape is (13, 617088)
Opening raw data file ../../../George Kenefati/Dan Friedman sEEG Data/Processed FIF Data/NY846_Day 8_Asleep2-raw.fif...
Isotrak not found
    Range : 0 ... 617087 =      

ValueError: all input arrays must have the same shape

In [24]:
raw = mne.io.read_raw_edf('../../../George Kenefati/Dan Friedman sEEG Data/NY846/Day 1/Asleep2/Asleep2.EDF')
raw_df = raw.to_data_frame(picks = selected_picks)
print(raw_df.shape)

Extracting EDF parameters from /media/sb10flpc002/08e63286-43ce-4f61-9491-1ed048c96f20/George Kenefati/Dan Friedman sEEG Data/NY846/Day 1/Asleep2/Asleep2.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
(615040, 14)


In [11]:
# GENERAL SETTINGS:

# 1. Choose Asleep or Awake data
sub_state = 'Asleep'

# 2. Set number of CPU cores to use for computing PSD. Default to -1 unless another user is running an intense program
n_cores = -1 # 1

# 3. Set maximum frequency for MUA 
MUA_max = 500

# 4. Set starting and ending times (in seconds)
tmin, tmax = 0, 300

# 5. Add in all subject IDs to be analyzed
sub_ids_lst = ['846', '853', '865', '870', '871', '872', '878', '884', '888', '893']


In [12]:
# data directories
processed_info_dir = f"../../../George Kenefati/Dan Friedman sEEG Data/Processed FIF Data/"
parent_data_dir = f"../../../George Kenefati/Dan Friedman sEEG Data/Processed FIF Data/"
save_dir = f"../../../George Kenefati/Dan Friedman sEEG Data/PO1 vs last day stats results/"
processed_roi_dir = f"../../../George Kenefati/Dan Friedman sEEG Data/Processed Anatomical Regions Data/"
# Globals
Fs = 2000 # Hz

bmin,bmax = 0,0 # baseline start and end for z-score

# TODO: change to appropriate labels for sEEG data. those below are from source space data
roi_names = [# Right
'rostralanteriorcingulate-rh', # Right Rostral ACC
'caudalanteriorcingulate-rh', # Right Caudal ACC
'postcentral-rh', # , Right S1
'ctx-rh-insula', 'superiorfrontal-rh', # Right Insula, Right DL-PFC
'medialorbitofrontal-rh', # Right Medial-PFC
# Left
 'rostralanteriorcingulate-lh', # Left Rostral ACC
 'caudalanteriorcingulate-lh', # Left Caudal ACC
 'postcentral-lh', # Left S1,
 'ctx-lh-insula', 'superiorfrontal-lh', # Left Insula, Left DL-PFC,
 'medialorbitofrontal-lh' # Left Medial-PFC
]

n_channels = len(roi_names)

In [13]:
# List of focus areas:
# caudalanteriorcingulate
# rostralanteriorcingulate
# ctx-lh-insula 
# ctx-rh-insula 
# medialorbitofrontal
# postcentral (S1)
# superiorfrontal (PFC)

# Of interest areas:
areas_of_interest = ['caudalanteriorcingulate','rostralanteriorcingulate','ctx-lh-insula','ctx-rh-insula','medialorbitofrontal','postcentral','superiorfrontal','insula']

# display.clear_output(wait=True)

In [14]:
first_day_data, last_day_data = file_pair_generator(processed_info_dir, sub_ids_lst[0], sub_state)


selected_picks = ROI_Depth_Generator(processed_roi_dir, sub_ids_lst[0])

In [15]:
for i in range(0, len(selected_picks)):
    if selected_picks[i][-2:][:-1] == '0':
        selected_picks[i] = selected_picks[i][:-2] + selected_picks[i][-1:]

In [16]:
selected_picks

['DLI5',
 'DLI6',
 'DLI7',
 'DLI8',
 'DLI9',
 'DRI1',
 'DRI2',
 'DRI3',
 'DRI4',
 'DRI5',
 'DRI6',
 'DRI7',
 'DRI8']

In [24]:
raw = mne.io.read_raw_fif(os.path.join(processed_info_dir, 'NY846_Day 1_Asleep1-raw.fif')).crop(tmin, tmax).load_data()
raw = raw.notch_filter(np.arange(120, 481, 60), n_jobs=-1, fir_design='firwin', notch_widths = 6) # inital notch filter for all harmonics
epochs = mne.make_fixed_length_epochs(raw, duration=30, preload=False)
n_fft = 2048  # the FFT size (n_fft). Ideally a power of 2

Opening raw data file ../../../George Kenefati/Dan Friedman sEEG Data/Processed FIF Data/NY846_Day 1_Asleep1-raw.fif...
Isotrak not found
    Range : 0 ... 617087 =      0.000 ...   301.312 secs
Ready.
Reading 0 ... 614400  =      0.000 ...   300.000 secs...
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 13517 samples (6.600 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    1.4s


Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=-1)]: Done 191 out of 191 | elapsed:    3.4s finished


In [57]:
raw_df = raw.to_data_frame(picks = selected_picks)
raw_df.head()

,time,DLI5,DLI6,DLI7,DLI8,DLI9,DRI1,DRI2,DRI3,DRI4,DRI5,DRI6,DRI7,DRI8
0,0.000000,-1.236668e-13,-1.389134e-13,-4.235165e-14,-1.694066e-13,-1.219727e-13,1.439956e-14,1.118083e-13,2.710505e-14,1.677125e-13,-1.736418e-14,-4.235165e-15,-4.065758e-14,-4.065758e-14
1,0.000488,-2.430374e+01,-2.510319e+01,-2.527172e+01,-2.646931e+01,-2.484184e+01,-7.125326e+00,-7.385539e+00,-6.276301e+00,-2.353287e+00,-1.578070e+00,-2.794631e+00,-3.933598e+00,-2.874039e+00
2,0.000977,-3.712937e+01,-3.905014e+01,-3.722450e+01,-4.003176e+01,-3.814335e+01,-6.721862e+00,-7.743634e+00,-4.963594e+00,2.268919e+00,5.605273e+00,9.082315e-01,1.294372e-01,2.281464e+00
3,0.001465,-4.301261e+01,-4.630812e+01,-4.291965e+01,-4.578092e+01,-4.450783e+01,-2.917431e+00,-3.078541e+00,-4.112710e-01,8.795677e+00,1.646070e+01,7.910000e+00,7.525035e+00,1.028792e+01
4,0.001953,-6.406634e+01,-6.882520e+01,-6.631395e+01,-6.721487e+01,-6.648522e+01,-2.845243e+00,-1.015110e-01,4.815097e-01,1.029361e+01,2.246314e+01,1.194985e+01,1.084512e+01,1.351084e+01


In [58]:
raw_df = raw_df['DLI5']
raw_df.head()

0   -1.236668e-13
1   -2.430374e+01
2   -3.712937e+01
3   -4.301261e+01
4   -6.406634e+01
Name: DLI5, dtype: float64

(61440, 14)


In [ ]:
z_score_baseline(processed_info_dir, ztmin, ztmax, sub_ids_lst[0], 

In [ ]:
        data_epo=combined_data_trials
        print(data_epo.shape)
        data_epo_zscore=np.copy(data_epo)
        for i in range(data_epo.shape[0]): # for each epoch
            for j in range(data_epo.shape[1]): # for each channel
                # compute mean and std of baseline
                base_mean = np.mean(data_epo[i,j,:len_baseline_samples])
                base_std = np.std(data_epo[i,j,:len_baseline_samples])

                # compute z-scored data from baseline stats
                data_epo_zscore[i,j,:] = (data_epo[i,j,:]-base_mean)/base_std

In [7]:
df_mean_all=[]
df_peak_all=[]
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ LOAD PROCESSED INFO @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ #
    

    # # Load in pain rating for each stimuli - pain ratings from Intracranial subjects are WIP
    # pain_ratings = scio.loadmat(os.path.join(processed_info_dir,sub_num+'_pain_ratings.mat'))
    # pain_ratings = pain_ratings['pain_ratings'].tolist()[0]
    # print(f"*pain_ratings.size = {len(pain_ratings)}*\n")

    # # Load in drop log for bad trials
    # drop_log = scio.loadmat(os.path.join(processed_info_dir,sub_num+'_drop_log.mat'))
    # drop_log = drop_log['drop_log'] # leave as array
    # print(f"*drop_log.size = {drop_log.shape[0]}*\n")

        display.clear_output(wait=True)

    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ COMPUTE PSD FROM RAW FIF @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ #

    # TODO: implement following tutorial for psd from raw.fif
    # https://mne.tools/0.21/auto_examples/time_frequency/plot_compute_raw_data_spectrum.html
    # done

    
    # Extract PSD into each frequency band of interest
    broad1 = epochs.compute_psd(fmin=1., fmax=55., n_jobs=-1) 
    broad2 = epochs.compute_psd(fmin=65., fmax=MUA_max., n_jobs=-1) # avoid the notch filter
    delta = epochs.compute_psd(fmin=1., fmax=4., n_jobs=-1)
    theta = epochs.compute_psd(fmin=4., fmax=8., n_jobs=-1)
    alpha = epochs.compute_psd(fmin=8., fmax=13., n_jobs=-1)
    beta = epochs.compute_psd(fmin=13., fmax=30., n_jobs=-1)
    low_gamma = epochs.compute_psd(fmin=30., fmax=55., n_jobs=-1)
    high_gamma = epochs.compute_psd(fmin=65., fmax=100., n_jobs=-1)
    MUA = fmin = 100, fmax = MUA_max
    
    # Get data (shape = epochs x channels x powers in each band (1 hz width))
    psds_broad1 = broad1.get_data()
    psds_broad2 = broad2.get_data()
    psds_delta = delta.get_data()
    psds_theta = theta.get_data()
    psds_alpha = alpha.get_data()
    psds_beta = beta.get_data() 
    psds_low_gamma = low_gamma.get_data()
    psds_high_gamma = high_gamma.get_data()

    # Get average power across bands (shape = epochs x channels)
    avg_psd_broad1 = np.mean(psds_broad1, axis=2)
    avg_psd_broad2 = np.mean(psds_broad1, axis=2)
    avg_psd_delta = np.mean(psds_delta, axis=2)
    avg_psd_theta = np.mean(psds_theta, axis=2)
    avg_psd_alpha = np.mean(psds_alpha, axis=2)
    avg_psd_beta = np.mean(psds_beta, axis=2)
    avg_psd_low_gamma = np.mean(psds_low_gamma, axis=2)
    avg_psd_high_gamma = np.mean(psds_high_gamma, axis=2)

    # Combine broad spectrum powers and take the trace
    avg_psd_broad = np.mean( np.array([avg_psd_broad1, avg_psd_broad2 ]), axis=0 )
    trace_avg_psd_broad = np.trace(avg_psd_broad)
    print("Trace of the average broad spectrum power: ",trace_avg_psd_broad)

    # Get band PSD's normalized by the trace of the broad spectrum and convert to dB
    avg_psd_delta = 10 * np.log10(avg_psd_delta / trace_avg_psd_broad)
    avg_psd_theta = 10 * np.log10(avg_psd_theta / trace_avg_psd_broad)
    avg_psd_alpha = 10 * np.log10(avg_psd_alpha / trace_avg_psd_broad)
    avg_psd_beta = 10 * np.log10(avg_psd_beta / trace_avg_psd_broad)
    avg_psd_low_gamma = 10 * np.log10(avg_psd_low_gamma / trace_avg_psd_broad)
    avg_psd_high_gamma = 10 * np.log10(avg_psd_high_gamma / trace_avg_psd_broad)

    avg_psd_arr = np.array((avg_psd_delta, avg_psd_theta, 
                            avg_psd_alpha, avg_psd_beta,
                            avg_psd_low_gamma, avg_psd_high_gamma))
    
    # # TODO: you may or may not need to use the following code block but im leaving it here just in case 
    # data_chs_array = np.zeros((len(roi_names),len(epo_times),Fs))
    # data_trials_tmp=np.zeros((len(epo_times),Fs))
    # for j,raw in enumerate(raw_objects_lst): # for each roi
    #     extracted_data_tmp=raw.data
    #     for i in range(len(epo_times)): # for each trial
    #         #all vertices
    #         all_vertices_tmp=extracted_data_tmp[:,int(epo_times[i]+tmin*Fs):int(epo_times[i]+tmax*Fs)]
    #         #averaged vertices
    #         vertices_averaged_tmp=np.mean(all_vertices_tmp,axis=0)
    #         #store trials x time data in numpy array
    #         data_trials_tmp[i,:] = vertices_averaged_tmp
    #     data_chs_array[j,...] = data_trials_tmp
    # # data in Array format 
    # data_chs_corrected_array=np.transpose(data_chs_array,(1,0,2))

    # Z-score data by for loop
    # TODO: z-score the data on a segment of the last-day data, and exclude that segment from the last-day data to be analyzed
    data_epo=combined_data_trials
    print(data_epo.shape)
    data_epo_zscore=np.copy(data_epo)
    for i in range(data_epo.shape[0]): # for each epoch
        for j in range(data_epo.shape[1]): # for each channel
            # compute mean and std of baseline
            base_mean = np.mean(data_epo[i,j,:len_baseline_samples])
            base_std = np.std(data_epo[i,j,:len_baseline_samples])
        
            # compute z-scored data from baseline stats
            data_epo_zscore[i,j,:] = (data_epo[i,j,:]-base_mean)/base_std

    # compare data before and after z-score
    print(data_epo[0,0,len_baseline_samples:len_baseline_samples+10],'\n')
    print(data_epo_zscore[0,0,len_baseline_samples:len_baseline_samples+10])

    # TODO: epochs object will just have z-scored day 1 data and last day data
    # Create info object required for epochs object
    info = mne.create_info(roi_names, sfreq=Fs,ch_types='eeg')
    data_epo = data_chs_corrected_array
    # Create events array for Epochs object
    # TODO: 0*i means the start time from each piece of data will just be 0
    # because we dont have any epochs in the data, we are treating the entire thing
    # as one epoch.
    # NOTE: the i in the last term of the list should just populate 0 and then 1,
    # indicating day 1 as label of 0 and last day as label of 1.
    
    events=np.array([[0*i,0,i] for i in range(len(conditions))])

    zepochs = mne.EpochsArray(data=data_epo_zscore,
                              info=info,
                              tmin=tmin,
                              events=events,
                              event_id=event_ids_dict,
                              baseline=(None,bmax),
                              )
    print(zepochs)
    epochs=zepochs 
    del zepochs

    display.clear_output(wait=True)
    
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ CONSTRUCT TFR OBJECT FOR DF @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ #
    
    # TODO: do not implement ERDS, leaving this code here to figure out how to use it to set up the dataframe
    tfr = tfr_multitaper(
        epochs,
        freqs=freqs,
        n_cycles=freqs,
        use_fft=True,
        return_itc=False,
        average=False,
        decim=2,
    )
    tfr.crop(tmin, tmax).apply_baseline(baseline, mode="percent")

    for event in event_ids:
        # select desired epochs for visualization
        tfr_ev = tfr[event]
        fig, axes = plt.subplots(
            1, 4, figsize=(12, 4), gridspec_kw={"width_ratios": [10, 10, 10, 1]}
        )
        for ch, ax in enumerate(axes[:-1]):  # for each channel
            # positive clusters
            _, c1, p1, _ = pcluster_test(tfr_ev.data[:, ch], tail=1, **kwargs)
            # negative clusters
            _, c2, p2, _ = pcluster_test(tfr_ev.data[:, ch], tail=-1, **kwargs)

            # note that we keep clusters with p <= 0.05 from the combined clusters
            # of two independent tests; in this example, we do not correct for
            # these two comparisons
            c = np.stack(c1 + c2, axis=2)  # combined clusters
            p = np.concatenate((p1, p2))  # combined p-values
            mask = c[..., p <= 0.05].any(axis=-1)

            # plot TFR (ERDS map with masking)
            tfr_ev.average().plot(
                [ch],
                cmap="RdBu",
                cnorm=cnorm,
                axes=ax,
                colorbar=False,
                show=False,
                mask=mask,
                mask_style="mask",
            )

            ax.set_title(epochs.ch_names[ch], fontsize=10)
            ax.axvline(0, linewidth=1, color="black", linestyle=":")  # event
            if ch != 0:
                ax.set_ylabel("")
                ax.set_yticklabels("")
        fig.colorbar(axes[0].images[-1], cax=axes[-1]).ax.set_yscale("linear")
        fig.suptitle(f"ERDS ({event})")
        plt.show()

    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ SET UP PANDAS DATAFRAMES @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ #

    #Export tfr to pandas df in seconds
    df = tfr.to_data_frame(time_format=None, long_format=False)
    df.head()

    #Plot with confidence bands 
    df = tfr.to_data_frame(time_format=None, long_format=True)

    # Map to frequency bands:
    freq_bounds = {'_': 0,
                   'delta': 4,
                   'theta': 7,
                   'alpha': 13,
                   'beta': 35,
                   'low-gamma': 55,
                   'notch': 65,
                   'high-gamma': 100,
                   'MUA': 500
                  }

    df['band'] = pd.cut(df['freq'], list(freq_bounds.values()),
                        labels=list(freq_bounds)[1:])

    # Filter to retain only relevant frequency bands:
    freq_bands_of_interest = [
                              # 'delta', 
                              'theta', 
                              'alpha', 
                              'beta', 
                              'low-gamma', 
                              'high-gamma',
                              'MUA',
        ]
    df = df[df.band.isin(freq_bands_of_interest)]
    df['band'] = df['band'].cat.remove_unused_categories()

    # Order channels for plotting:
    df['channel'] = df['channel'].cat.reorder_categories(tuple(roi_names),
                                                         ordered=True)

    g = sns.FacetGrid(df, row='band', col='channel', margin_titles=True)
    g.map(sns.lineplot, 'time', 'value', n_boot=10)
    axline_kw = dict(color='black', linestyle='dashed', linewidth=0.5, alpha=0.5)
    g.map(plt.axhline, y=0, **axline_kw)
    g.map(plt.axvline, x=0, **axline_kw)
    g.set(xlim=(tmin+0.5,tmax-0.5))
    # g.set(ylim=(None, 3))
    g.set_axis_labels("Time (s)", "Z-scored Band Power (uV^2)")
    g.set_titles(col_template="{col_name}", row_template="{row_name}")
    g.add_legend(ncol=2, loc='lower center')
    g.fig.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.08)
    plt.show();

    display.clear_output(wait=True)

    # Plot the ERDS based on average, peak amplitude, and area-under-the-curve (AUC), one by one.
    df_mean = (df.query(f"time > {tmin}")
                 .groupby(['epoch','band','channel'])[['value']]
                 .mean()
                 .dropna(subset=['value']) # needed to remove taking average over non-numeric values
                 .reset_index())

    df_mean_all.append(df_mean) #when it loops through each subject, will save the mean in the array

    df_peak = (df.query(f"time > {tmin} & time < {tmax}")
                 .groupby(['epoch','band','channel'])[['value']]
                 .max() # peak amplitude
                 .dropna(subset=['value']) # needed to remove taking average over non-numeric values
                 .reset_index())
    df_peak_all.append(df_peak) #when it loops through each subject, will save the peak

    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ EVALUATE AND SAVE STATS OUTPUTS @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ #
    stats_plot(df_mean, save_dir, 'mean')
    stats_plot(df_peak, save_dir, 'peak')
    
    display.clear_output(wait=True)

IndentationError: unexpected indent (1711115917.py, line 30)